# Libraries

In [1]:
from tqdm.notebook import tqdm
import torch
print("Using torch", torch.__version__)
import torch.nn as nn
import torch.nn.functional as F

Using torch 2.0.0+cpu


In [2]:
import torchvision

In [3]:
print("Using torch", torchvision.__version__)

Using torch 0.14.0+cpu


In [4]:
import os.path
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import pandas as pd

# dataset

In [5]:
folder = r"C:\Users\Isma Imtiaz\OneDrive\Desktop\sentiment_images - Copy"

In [6]:
file = r"sentiment|.csv"

# cleaning datasets

In [7]:
gh = pd.read_csv("sentiment.csv")


In [8]:
gh

,Unnamed: 0,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
0,0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.
1,1,31369,train,COCO_val2014_000000389081.jpg,1,"['french', 'fries', 'are', 'not', 'a', 'health...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...
2,2,31369,train,COCO_val2014_000000389081.jpg,1,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"The plate has one of my favorite foods on it, ..."
3,3,31369,train,COCO_val2014_000000389081.jpg,1,"['it', 'was', 'disgusting', 'food', 'not', 'ju...","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"It was disgusting food, not just bad food."
4,4,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,A plate of disgusting food found at a diner.
...,...,...,...,...,...,...,...,...,...
39194,39194,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...
39195,39195,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,A dirty bathroom that has a window in it.
39196,39196,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom
39197,39197,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...


In [9]:
test = gh[gh['split'] == 'test']

In [10]:
test

,Unnamed: 0,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
23251,23251,26123,test,COCO_val2014_000000497106.jpg,1,"['a', 'person', 'wearing', 'a', 'beautiful', '...","[0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0,...",1,a person wearing a beautiful dress under an um...
23252,23252,26123,test,COCO_val2014_000000497106.jpg,1,"['a', 'person', 'with', 'a', 'beautiful', 'dre...","[0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0,...",1,A person with a beautiful dress with an umbrel...
23253,23253,26123,test,COCO_val2014_000000497106.jpg,1,"['the', 'adorable', 'child', 'wears', 'a', 'pr...","[0.0, 1, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0...",1,"The adorable child wears a pretty dress, that ..."
23254,23254,26123,test,COCO_val2014_000000497106.jpg,1,"['a', 'young', 'shy', 'girl', 'in', 'a', 'fiar...","[0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0,a young shy girl in a fiary dress under an umb...
23255,23255,26123,test,COCO_val2014_000000497106.jpg,1,"['a', 'frightened', 'child', 'wearing', 'a', '...","[0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0,a frightened child wearing a dress with wings ...
...,...,...,...,...,...,...,...,...,...
39194,39194,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...
39195,39195,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,A dirty bathroom that has a window in it.
39196,39196,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom
39197,39197,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...


In [11]:
tokens1 = test['tokens']

In [12]:
train = gh[gh['split'] == 'train']

In [13]:
train

,Unnamed: 0,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
0,0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.
1,1,31369,train,COCO_val2014_000000389081.jpg,1,"['french', 'fries', 'are', 'not', 'a', 'health...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...
2,2,31369,train,COCO_val2014_000000389081.jpg,1,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"The plate has one of my favorite foods on it, ..."
3,3,31369,train,COCO_val2014_000000389081.jpg,1,"['it', 'was', 'disgusting', 'food', 'not', 'ju...","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"It was disgusting food, not just bad food."
4,4,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,A plate of disgusting food found at a diner.
...,...,...,...,...,...,...,...,...,...
23246,23246,16855,train,COCO_val2014_000000575776.jpg,1,"['a', 'stupid', 'animal', 'and', 'a', 'rhino',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0,A stupid animal and a rhino interacting in the...
23247,23247,20238,train,COCO_val2014_000000464358.jpg,1,"['dead', 'light', 'from', 'the', 'window', 're...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1,...",0,Dead light from the window reveals a once comf...
23248,23248,20238,train,COCO_val2014_000000464358.jpg,1,"['the', 'thieves', 'used', 'the', 'broken', 'w...","[0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,The thieves used the broken window to enter th...
23249,23249,20238,train,COCO_val2014_000000464358.jpg,1,"['dead', 'light', 'from', 'the', 'window', 're...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1,...",0,Dead light from the window reveals a once comf...


In [14]:
word_sentiment_train = train['word_sentiment']

In [15]:
labels_train = train['word_sentiment']

In [16]:
labels_train[0]

'[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]'

# pentagram

In [17]:
train['tokens'] = train['tokens'].apply(lambda x: x.strip("[]").replace("'", "").split(", "))
new_tokens = []
for row in train['tokens']:
    new_row = [0, 0] + row + [0, 0]
    new_tokens.append(new_row)


C:\Users\Isma Imtiaz\AppData\Local\Temp\ipykernel_20836\770442965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['tokens'] = train['tokens'].apply(lambda x: x.strip("[]").replace("'", "").split(", "))


In [18]:
len(new_tokens)

19325

# training 

# inserting all pentagram in list and making in string format

In [19]:
import nltk
pentagrams = []
for tokens in new_tokens:
    pentagrams.append(list(nltk.ngrams(tokens, 5, pad_left=False, pad_right=False)))

In [20]:
len(pentagrams)

19325

In [21]:
pentagram_tuples = [pent for sublist in pentagrams for pent in sublist]

In [22]:
pentagram_tuples

[(0, 0, 'a', 'plate', 'of'),
 (0, 'a', 'plate', 'of', 'delicious'),
 ('a', 'plate', 'of', 'delicious', 'food'),
 ('plate', 'of', 'delicious', 'food', 'including'),
 ('of', 'delicious', 'food', 'including', 'french'),
 ('delicious', 'food', 'including', 'french', 'fries'),
 ('food', 'including', 'french', 'fries', 0),
 ('including', 'french', 'fries', 0, 0),
 (0, 0, 'french', 'fries', 'are'),
 (0, 'french', 'fries', 'are', 'not'),
 ('french', 'fries', 'are', 'not', 'a'),
 ('fries', 'are', 'not', 'a', 'healthy'),
 ('are', 'not', 'a', 'healthy', 'food'),
 ('not', 'a', 'healthy', 'food', 'but'),
 ('a', 'healthy', 'food', 'but', 'it'),
 ('healthy', 'food', 'but', 'it', 'is'),
 ('food', 'but', 'it', 'is', 'an'),
 ('but', 'it', 'is', 'an', 'excellent'),
 ('it', 'is', 'an', 'excellent', 'food'),
 ('is', 'an', 'excellent', 'food', 'for'),
 ('an', 'excellent', 'food', 'for', 'teenagers'),
 ('excellent', 'food', 'for', 'teenagers', 0),
 ('food', 'for', 'teenagers', 0, 0),
 (0, 0, 'the', 'plate', 

In [23]:
len(pentagram_tuples)

229205

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
docs = [str(t[1]) for t in pentagram_tuples]

In [25]:
docs

['0',
 'a',
 'plate',
 'of',
 'delicious',
 'food',
 'including',
 'french',
 '0',
 'french',
 'fries',
 'are',
 'not',
 'a',
 'healthy',
 'food',
 'but',
 'it',
 'is',
 'an',
 'excellent',
 'food',
 'for',
 '0',
 'the',
 'plate',
 'has',
 'one',
 'of',
 'my',
 'favorite',
 'foods',
 'on',
 'it',
 'french',
 '0',
 'it',
 'was',
 'disgusting',
 'food',
 'not',
 'just',
 'bad',
 '0',
 'a',
 'plate',
 'of',
 'disgusting',
 'food',
 'found',
 'at',
 'a',
 '0',
 'the',
 'meat',
 'on',
 'the',
 'burger',
 'looks',
 'like',
 'disgusting',
 '0',
 'a',
 'plate',
 'of',
 'delicious',
 'food',
 'including',
 'french',
 '0',
 'french',
 'fries',
 'are',
 'not',
 'a',
 'healthy',
 'food',
 'but',
 'it',
 'is',
 'an',
 'excellent',
 'food',
 'for',
 '0',
 'the',
 'plate',
 'has',
 'one',
 'of',
 'my',
 'favorite',
 'foods',
 'on',
 'it',
 'french',
 '0',
 'it',
 'was',
 'disgusting',
 'food',
 'not',
 'just',
 'bad',
 '0',
 'a',
 'plate',
 'of',
 'disgusting',
 'food',
 'found',
 'at',
 'a',
 '0',
 

# tfidf vectorizer in pentagram tuples

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
docs = [str(t[1]) for t in pentagram_tuples]

tfidf_vectorizer = TfidfVectorizer()
placed = tfidf_vectorizer.fit(tokens1)
method_of = tfidf_vectorizer.transform(docs)
textofsentiment = torch.tensor(method_of.toarray(), dtype=torch.float32)

In [27]:
textofsentiment.shape

torch.Size([229205, 2884])

# model

In [50]:
class TextSimpleClassifier(nn.Module):
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()
        self.relu_fn = nn.ReLU()
        self.act_fn = nn.Tanh()
        self.hidden_layers = nn.Sequential(
            nn.Linear(num_inputs, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, num_hidden),
            nn.ReLU(),
        )
        self.output_layer = nn.Linear(num_hidden, num_outputs)

    def forward(self, x):
        x = self.hidden_layers(x)
        x = self.act_fn(x)
        x = self.output_layer(x)
        x = self.relu_fn(x)
        return x

text_model = TextSimpleClassifier(2884, 100, 1)


In [51]:
labelofim = [[float(label) for label in row.strip('][').split(', ')] for row in train['word_sentiment']]

In [52]:
max_len = max(len(row) for row in labelofim)
labelofim = [row + [0]*(max_len-len(row)) for row in labelofim]
labelofim = [[float(label) for label in row] for row in labelofim]
labeloftext = torch.tensor(labelofim)

In [53]:
labeloftext.shape

torch.Size([19325, 39])

# dataset

In [54]:
import torch
from torch.utils.data import Dataset, DataLoader


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, text_data, labels,x):
        self.text_data = text_data
        for i in labels:
            print(i)
        self.labels = labels

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        idx_text = idx % len(self.text_data)
        idx_label = idx % len(self.labels)
        label = self.labels[idx_label]
        text = self.text_data[idx_text]
        return text, label.float()


Dataset = MyDataset(textofsentiment,labeloftext,5)
batch_size = 10
trainloader = DataLoader(Dataset, batch_size=batch_size, shuffle=True)

In [55]:
Dataset

In [56]:
trainloader

In [57]:
for i in trainloader:
    print(i[0].shape)
    break

torch.Size([10, 2884])


In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
criterion = nn.MSELoss()
optimizer = optim.SGD(text_model.parameters(), lr=0.01)
num_correct = 0
num_total = 0
input_size = 10

for textdata, labeldata in trainloader:
    print(textdata.shape)
    print(label.shape)
    labeldata = labeldata.float()
    m2 = text_model(textdata)
    output = text_model(textofsentiment)
    predicted = torch.round(output)
    for i in predicted:
        y = i + 2
        print(i)
    num_correct += (predicted == labeldata.unsqueeze(1)).sum().item()
    num_total += labeldata.size(0)

accuracy = num_correct / num_total
print(f"Train accuracy: {accuracy/100:.4f}")

Train accuracy: 75.04


# Testing

In [71]:
test

,Unnamed: 0,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
23251,23251,26123,test,COCO_val2014_000000497106.jpg,1,"['a', 'person', 'wearing', 'a', 'beautiful', '...","[0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0,...",1,a person wearing a beautiful dress under an um...
23252,23252,26123,test,COCO_val2014_000000497106.jpg,1,"['a', 'person', 'with', 'a', 'beautiful', 'dre...","[0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0,...",1,A person with a beautiful dress with an umbrel...
23253,23253,26123,test,COCO_val2014_000000497106.jpg,1,"['the', 'adorable', 'child', 'wears', 'a', 'pr...","[0.0, 1, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0...",1,"The adorable child wears a pretty dress, that ..."
23254,23254,26123,test,COCO_val2014_000000497106.jpg,1,"['a', 'young', 'shy', 'girl', 'in', 'a', 'fiar...","[0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0,a young shy girl in a fiary dress under an umb...
23255,23255,26123,test,COCO_val2014_000000497106.jpg,1,"['a', 'frightened', 'child', 'wearing', 'a', '...","[0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0,a frightened child wearing a dress with wings ...
...,...,...,...,...,...,...,...,...,...
39194,39194,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...
39195,39195,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,A dirty bathroom that has a window in it.
39196,39196,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom
39197,39197,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...


In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
placed = tfidf_vectorizer.fit(test)
method_of = tfidf_vectorizer.transform(test['tokens'])
textofsentiment = torch.tensor(method_of.toarray(), dtype=torch.float32)

In [75]:
textofsentiment.shape

torch.Size([15948, 9])

In [76]:
labels = test['word_sentiment']

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
criterion = nn.MSELoss()
optimizer = optim.SGD(text_model.parameters(), lr=0.01)
num_correct = 0
num_total = 0
input_size = 10

for textdata, labeldata in trainloader:
    labeldata = labeldata.float()
    m2 = text_model(textofsentiment)
    output = text_model(textofsentiment)
    predicted = torch.round(output)
    for i in predicted:
        y = i + 2
        print(i)
    num_correct += (predicted == labeldata.unsqueeze(1)).sum().item()
    num_total += labeldata.size(0)

accuracy = num_correct / num_total
print(f"Test accuracy: {accuracy/100:.4f}")

Epoch [1/10], Step [100/22921], Loss: -9.4386
Epoch [1/10], Step [200/22921], Loss: -25.1299
Epoch [1/10], Step [300/22921], Loss: -74.2425
Epoch [1/10], Step [400/22921], Loss: -42.0554
Epoch [1/10], Step [500/22921], Loss: -103.9888
Epoch [1/10], Step [600/22921], Loss: -149.6882
Epoch [1/10], Step [700/22921], Loss: -110.7082
Epoch [1/10], Step [800/22921], Loss: -58.1878
Epoch [1/10], Step [900/22921], Loss: -180.8349
Epoch [1/10], Step [1000/22921], Loss: -177.6357
Epoch [1/10], Step [1100/22921], Loss: -251.1320
Epoch [1/10], Step [1200/22921], Loss: -147.8508
Epoch [1/10], Step [1300/22921], Loss: -184.8954
Epoch [1/10], Step [1400/22921], Loss: -107.4183
Epoch [1/10], Step [1500/22921], Loss: -220.9207
Epoch [1/10], Step [1600/22921], Loss: -372.3165
Epoch [1/10], Step [1700/22921], Loss: -232.0339
Epoch [1/10], Step [1800/22921], Loss: -393.3957
Epoch [1/10], Step [1900/22921], Loss: -141.7596
Epoch [1/10], Step [2000/22921], Loss: -356.7178
Epoch [1/10], Step [2100/22921], Lo